In [801]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/flietest/Test.xlsx
/kaggle/input/trains/Train.xlsx


In [802]:
df = pd.read_excel('/kaggle/input/trains/Train.xlsx')
df = df.set_index('NO')
df.head()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q69,Q70,Q71,Q72,Q73,Q74,Q75,Q76,Q77,CLASS
NO,,,,,,,,,,,,,,,,,,,,,
11,1,5,2,5,1,3,1,1,1,2,...,4,3,1,4,1,2,2,3,4,1
12,2,5,1,5,1,5,1,1,1,2,...,5,3,1,5,1,2,1,3,1,1
13,2,4,2,5,2,5,2,2,2,2,...,4,3,2,3,2,3,2,2,4,1
14,2,5,1,5,2,5,2,1,2,2,...,3,2,2,4,2,2,4,3,4,1
15,1,4,1,5,1,3,1,1,1,1,...,4,2,2,3,1,3,1,3,5,1


In [803]:
df.shape

(281, 78)

In [804]:
A = df.isnull().sum(axis=0)
A

Q1       0
Q2       0
Q3       0
Q4       0
Q5       0
        ..
Q74      0
Q75      0
Q76      0
Q77      0
CLASS    0
Length: 78, dtype: int64

In [805]:
nan = np.array(A)
nan

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [806]:
df_clean = df.dropna(1,'any')

In [807]:
B = df_clean.isnull().sum(axis=0)
B

Q1       0
Q2       0
Q3       0
Q4       0
Q5       0
        ..
Q74      0
Q75      0
Q76      0
Q77      0
CLASS    0
Length: 77, dtype: int64

In [808]:
nan = np.array(B)
nan

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [809]:
pel_class = np.unique(df_clean['CLASS'])
pel_class

array([1, 2, 3])

In [810]:
pel_value = [sum(df_clean['CLASS']==pel_class[0]),
             sum(df_clean['CLASS']==pel_class[1]),
             sum(df_clean['CLASS']==pel_class[2])]
pel_value

[134, 86, 61]

In [811]:
y = df_clean['CLASS'].values
X = df_clean.drop(['CLASS'],axis = 1).values

In [812]:
X = np.array(X)
y = np.array(y)

In [813]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [814]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

In [815]:
X_train_norm.shape

(224, 76)

In [816]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=1)
X_train_norm, y_train = sm.fit_resample(X_train_norm, y_train)

In [817]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
fs = SelectKBest(chi2, k=50)
fs.fit(X_train_norm, y_train)
X_train_norm = fs.transform(X_train_norm)
X_test_norm = fs.transform(X_test_norm)

In [818]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
params = {
    'C' : [1,2,4,8,16], 
    'gamma' : [0.03125, 0.0625, 0.125, 0.25, 0.5, 1, 2, 4, 8, 16, 32] # High gamma = Overfitting
}
clf = GridSearchCV(SVC(),params, cv=10)
clf.fit(X_train_norm, y_train)
print("Best params : " + str(clf.best_params_))
print("10CV accuracy : "+str(clf.best_score_*100))

NameError: name 'X_norm' is not defined

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
y_predict = clf.predict(X_test_norm)
print("Test accuracy : "+str(sum(y_test == y_predict)/len(y_test)*100))

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
target_names = ['negative', 'positive']
C = confusion_matrix(y_test,y_predict) 
C = C / C.astype(np.float).sum(axis=1)*100
sns.heatmap(C, annot=True, fmt="f",cmap="GnBu",xticklabels=target_names, yticklabels=target_names)
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.show()

In [ ]:
X_test_norm.shape

# Test

In [ ]:
df = pd.read_excel('/kaggle/input/flietest/Test.xlsx')
df = df.set_index('NO')

In [ ]:
df_clean = df.drop(['Q31'],axis = 1)

In [ ]:
yt = df_clean['CLASS'].values
Xt = df_clean.drop(['CLASS'],axis = 1).values

In [ ]:
X_test = scaler.transform(Xt)
X_test_n = fs.transform(X_test)

In [ ]:
y_predict = clf.predict(X_test_n)
print("Test accuracy : "+str(sum(yt == y_predict)/len(yt)*100))